In [ ]:
from langchain.chat_models import ChatOpenAI
from getpass import getpass
from langchain.schema import HumanMessage, AIMessage, SystemMessage
import os
import json

In [ ]:
# setting up an OpenAI template on the run
OPENAI_API_KEY = getpass()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [ ]:
model = ChatOpenAI(model="gpt-4")
model.temperature = 0.8

In [ ]:
from datasets import load_dataset
all_data = []
dataset = load_dataset("THUDM/AgentInstruct", split="os")
for ds in dataset:
    all_data.append(ds)

In [ ]:
def convert_to_convs(s):
    all_convs = []
    s = s.split("## USER: Now, I will start a new problem in a new OS. My problem is:")
    s = [ss for ss in s if ss.strip()!="" ]
    s = ["## USER: Now, I will start a new problem in a new OS. My problem is:"+ss for ss in s]
    for ss in s:
        _dict = {"conversations": []}
        for sss in ss.split("## USER:")[1:]:
            user_conv, agent_conv = sss.split('## AGENT:')[0].strip(), sss.split('## AGENT:')[1].strip()
            _dict["conversations"] += [{"from": "human", "value": user_conv, "loss": False}, {"from": "gpt", "value": agent_conv, "loss": True}]
        all_convs.append(_dict)
    return all_convs

In [ ]:
from tqdm.auto import tqdm 
import random
num_turns = 1000
init_prompt = \
"""
Given is a conversations between an AI agent and a user. The user asks the agent to solve a task and in return the agent suggests the solution to that task. The agent produces bash commands which the user runs in his linux operating system and returns the output to the agent. You have to generate some more conversations following the previous. Pls be innovative in creating the task for the agent. 
"""
all_convs = []
all_tasks = []
for _ in tqdm(range(num_turns)):
    random.shuffle(all_data)
    prompt = ""
    for dd in all_data[0]['conversations'][:6]:
        _from = "USER" if dd['from']=="human" else "AGENT"
        prompt+=f"\n\n## {_from}: {dd['value']}"
        if "Now, I will start a new problem in a new OS. My problem is:\n\n" in dd["value"]:
            all_tasks.append(dd["value"].replace("Now, I will start a new problem in a new OS. My problem is:\n\n", ""))
    for d in all_data[1:7]:
        for dd in d["conversations"][6:]:
            _from = "USER" if dd['from']=="human" else "AGENT"
            prompt += f"\n\n## {_from}: {dd['value']}"
            if "Now, I will start a new problem in a new OS. My problem is:\n\n" in dd["value"]:
                all_tasks.append(dd["value"].replace("Now, I will start a new problem in a new OS. My problem is:\n\n", ""))
    task_messages =[
        SystemMessage(content=init_task_gen_prompt),
        HumanMessage(content="\n".join(all_tasks))
    ]
    messages = [
        SystemMessage(content=init_prompt),
        HumanMessage(content=prompt)
    ]
    try:
        response = model.predict_messages(messages)
        convs = convert_to_convs(response.content)
        all_convs.extend(convs)    
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass
    try:
        response = model.predict_messages(messages)
        convs = convert_to_convs(response.content)
        all_convs.extend(convs)
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass    
    try:
        response = model.predict_messages(messages)
        convs = convert_to_convs(response.content)
        all_convs.extend(convs)    
        messages.append(response)
    except:
        print("Not the assumed format!!") 
        pass

    with open('./synthetic-3/os_syn-datasyntype_2-2.json', 'w') as file:
        json.dump(all_convs, file)
        # for d in all_convs:
        #     json.dump(d, file)
        #     file.write('\n')
    # break